In [1]:
import matplotlib
import matplotlib.patches as patches
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from pathlib import Path
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter import messagebox
import math

#Eventos
NO_EV=0
QUIT_EV=1
TIME_FREQ_BUTTON_EV=2
PLOT_BUTTON_EV=3
INPUT_CHANGED_EV=4
GRAPH_BUTTON_EV= 5
ELLIPSE_EV = 6

#Largos y Anchos
GRAPH_WIDTH=1200
GRAPH_HEIGHT=400

#Colores
FRAME_COLOR= "goldenrod"
FRAME_TEXT_COLOR="black"
BUTTON_COLOR= "light goldenrod"
BUTTON_FONT_COLOR="black"
GRAPH_BUTTON_COLOR="Cyan1"
GRAPH_BUTTON_TEXT_COLOR="black"

#Filtros
FILTROS=[
    "ramp",
    "shepp-logan",
    "cosine",
    "hamming",
    "hann"
    ]
#Interpolaciones
INTERP=[
    "linear",
    "nearest",
    "cubic"]
#Nodos
A=1
B=2
INPUT=4

class SimGUI:
    """Clase que se encarga de manejar la interfaz grafic de la simulacion"""
    def __init__(self):
        self.graphed_once= False
        self.input_changed= False
        self.remove_ellipse = False
        self.add_ellipse = False

        self.Ev=NO_EV
        self.root = Tk()
        self.root.geometry('1620x780')
        self.root.resizable(width=True, height=True)
        self.root.title("Tomografia")
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

        self.CreatePlot()
        self.CreateOptions()

    def CreatePlot(self):
        #Region donde se grafican las funciones
        self.PlotFrame = LabelFrame(master=self.root, text="Grafica",background=FRAME_COLOR,fg=FRAME_TEXT_COLOR)
        self.PlotFrame.grid(row=0,column=1,sticky=N+S+W+E)
        self.fig=Figure(figsize=(15,15), dpi=50,facecolor="lavender",constrained_layout=False)
        self.Graph = FigureCanvasTkAgg(self.fig,master=self.PlotFrame)
        self.Graph.get_tk_widget().config( width=GRAPH_WIDTH, height=GRAPH_HEIGHT)
        self.Graph.get_tk_widget().grid(row=0)
        #Creo una toolbar para los graficos
        self.toolbarFrame = Frame(master=self.PlotFrame)
        self.toolbarFrame.grid(row=1)
        toolbar = NavigationToolbar2Tk(self.Graph, self.toolbarFrame)
        toolbar.grid(row=0)
        #Inicializo Axes
        self.InitializeAxes()
        #Creo region para botones
        self.SelectedGraph= tk.IntVar()
        self.GraphButtonFrame = Frame(master=self.PlotFrame)
        self.GraphButtonFrame.grid(row=2)
        #Boton para graficar
        self.GraphButton = Button(master=self.GraphButtonFrame,text="Graficar",background=GRAPH_BUTTON_COLOR,fg=GRAPH_BUTTON_TEXT_COLOR,command=self.graph_button_call)
        self.GraphButton.grid(row=0,column=0,sticky=W+E)
        
    def CreateOptions(self):
        #Region con los parametros posibles
        self.OptionsFrame = LabelFrame(master=self.root, text="Parametros",background=FRAME_COLOR,fg=FRAME_TEXT_COLOR)
        self.OptionsFrame.grid(row=100,sticky=W+E+S+N,columnspan=10,ipady=20)
        self.CreateInputSection()
        self.CreateFilterSection()
        self.CreateSamplerSection()

    def CreateInputSection(self):
        self.InputFrame = LabelFrame(master=self.OptionsFrame, text="Phantom",background=FRAME_COLOR,fg=FRAME_TEXT_COLOR)
        self.InputFrame.grid(row=0,column=0,sticky=S+N)
        #Intensity Slider
        self.SlideIntensity_Label = Label(master=self.InputFrame,text="Intensity",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideIntensity_Label.grid(row=0,column=0,sticky=N+S+W+E)
        self.SlideIntensity = Scale(master=self.InputFrame, from_=-1, to=1,orient=HORIZONTAL,resolution=0.01,command= self.input_change_callback)
        self.SlideIntensity.config(bg=BUTTON_COLOR)
        self.SlideIntensity.grid(row=0,column=1,sticky=W+E,ipadx=80)
        #Tilt Slider
        self.SlideTilt_Label = Label(master=self.InputFrame,text="Tilt",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideTilt_Label.grid(row=0,column=2,sticky=N+S+W+E)
        self.SlideTilt = Scale(master=self.InputFrame, from_=0, to=180,orient=HORIZONTAL,command= self.input_change_callback)
        self.SlideTilt.config(bg=BUTTON_COLOR)
        self.SlideTilt.grid(row=0,column=3,sticky=W+E,ipadx=80)
        #X Semi-Axis Slider
        self.SlideX_Axis_Label = Label(master=self.InputFrame,text="X Semi-axis",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideX_Axis_Label.grid(row=1,column=0,sticky=N+S+W+E)
        self.SlideX_Axis = Scale(master=self.InputFrame, from_=0.01, to=1,orient=HORIZONTAL,resolution=0.01,command= self.input_change_callback)
        self.SlideX_Axis.config(bg=BUTTON_COLOR)
        self.SlideX_Axis.grid(row=1,column=1,sticky=W+E,ipadx=80)
        #Y Semi-Axis Slider
        self.SlideY_Axis_Label = Label(master=self.InputFrame,text="Y Semi-axis",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideY_Axis_Label.grid(row=1,column=2,sticky=N+S+W+E)
        self.SlideY_Axis = Scale(master=self.InputFrame, from_=0.01, to=1,orient=HORIZONTAL,resolution=0.01,command= self.input_change_callback)
        self.SlideY_Axis.config(bg=BUTTON_COLOR)
        self.SlideY_Axis.grid(row=1,column=3,sticky=W+E,ipadx=80)
        #X Center Slider
        self.SlideX_Center_Label = Label(master=self.InputFrame,text="X Center",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideX_Center_Label.grid(row=2,column=0,sticky=N+S+W+E)
        self.SlideX_Center = Scale(master=self.InputFrame, from_=-1, to=1,orient=HORIZONTAL,resolution=0.01,command= self.input_change_callback)
        self.SlideX_Center.config(bg=BUTTON_COLOR)
        self.SlideX_Center.grid(row=2,column=1,sticky=W+E,ipadx=80)
        #Y Center Slider
        self.SlideY_Center_Label = Label(master=self.InputFrame,text="Y Center",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideY_Center_Label.grid(row=2,column=2,sticky=N+S+W+E)
        self.SlideY_Center = Scale(master=self.InputFrame, from_=-1, to=1,orient=HORIZONTAL,resolution=0.01,command= self.input_change_callback)
        self.SlideY_Center.config(bg=BUTTON_COLOR)
        self.SlideY_Center.grid(row=2,column=3,sticky=W+E,ipadx=80)
        #Botones
        self.AddEllipse_Button= Button(master=self.InputFrame,text="Add elipse",background=GRAPH_BUTTON_COLOR,fg=GRAPH_BUTTON_TEXT_COLOR,command=self.add_ellipse_button_call)
        self.AddEllipse_Button.grid(row=3,column=1,sticky=N+S+W+E,ipadx=40)
        self.RemoveEllipse_Button= Button(master=self.InputFrame,text="Remove Last Elipse",background=GRAPH_BUTTON_COLOR,fg=GRAPH_BUTTON_TEXT_COLOR,command=self.remove_ellipse_button_call)
        self.RemoveEllipse_Button.grid(row=3,column=3,sticky=N+S+W+E,ipadx=40)
        

    def CreateFilterSection(self):
        self.FilterFrame = LabelFrame(master=self.OptionsFrame, text="Radon Transform",background=FRAME_COLOR,fg=FRAME_TEXT_COLOR)
        self.FilterFrame.grid(row=0,column=1,sticky=S+N)
        #Start Slider
        self.SlideStart_Label = Label(master=self.FilterFrame,text="angle start(deg)",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideStart_Label.grid(row=0,column=0,sticky=N+S+W+E)
        self.SlideStart = Scale(master=self.FilterFrame, from_=0, to=180,orient=HORIZONTAL,command= self.input_change_callback)
        self.SlideStart.config(bg=BUTTON_COLOR)
        self.SlideStart.grid(row=0,column=1,sticky=W+E,ipadx=80)
        #End Slider
        self.SlideEnd_Label = Label(master=self.FilterFrame,text="angle end(deg)",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideEnd_Label.grid(row=1,column=0,sticky=N+S+W+E)
        self.SlideEnd = Scale(master=self.FilterFrame, from_=0, to=180,orient=HORIZONTAL,command= self.input_change_callback)
        self.SlideEnd.config(bg=BUTTON_COLOR)
        self.SlideEnd.grid(row=1,column=1,sticky=W+E,ipadx=80)
        #Step Slider
        self.SlideStep_Label = Label(master=self.FilterFrame,text="angle step(deg)",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlideStep_Label.grid(row=2,column=0,sticky=N+S+W+E)
        self.SlideStep = Scale(master=self.FilterFrame, from_=0.05, to=180,orient=HORIZONTAL,command= self.input_change_callback)
        self.SlideStep.config(bg=BUTTON_COLOR)
        self.SlideStep.grid(row=2,column=1,sticky=W+E,ipadx=80)
        #Plot Slider
        self.SlidePlotAngle_Label = Label(master=self.FilterFrame,text="angle to show(deg)",anchor=W,background=BUTTON_COLOR,fg=BUTTON_FONT_COLOR)
        self.SlidePlotAngle_Label.grid(row=3,column=0,sticky=N+S+W+E)
        self.SlidePlotAngle = Scale(master=self.FilterFrame, from_=0, to=180,orient=HORIZONTAL,command= self.input_change_callback)
        self.SlidePlotAngle.config(bg=BUTTON_COLOR)
        self.SlidePlotAngle.grid(row=3,column=1,sticky=W+E,ipadx=80)

    def CreateSamplerSection(self):
        self.SamplerFrame = LabelFrame(master=self.OptionsFrame, text="Reconstruction",background=FRAME_COLOR,fg=FRAME_TEXT_COLOR)
        self.SamplerFrame.grid(row=0,column=2,sticky=S+N)
        #Filtro a utilizar
        self.selected_filter = StringVar(master=self.SamplerFrame)
        self.selected_filter.set(FILTROS[0]) # Empieza con ramp como default
        self.filter_pull_down_menu = OptionMenu(self.SamplerFrame, self.selected_filter, *FILTROS)
        self.filter_pull_down_menu.config(bg=BUTTON_COLOR)
        self.filter_pull_down_menu.grid(row=0,sticky=W+E)
        #Interpolacion a utilizar
        self.selected_interp = StringVar(master=self.SamplerFrame)
        self.selected_interp.set(INTERP[0]) # Empieza con ramp como default
        self.interp_pull_down_menu = OptionMenu(self.SamplerFrame, self.selected_interp, *INTERP)
        self.interp_pull_down_menu.config(bg=BUTTON_COLOR)
        self.interp_pull_down_menu.grid(row=1,sticky=W+E)

    #Getters
    def GetEvent(self):
        return self.Ev
    def GetSelectedDomain(self):
        return (self.SelectedGraph.get())
    def GetSelectedFunc(self):
        return self.selected_func.get()

    #Funciones de la grafica
    def InitializeAxes(self):
        self.AxesInput= self.fig.add_subplot(221)
        self.AxesRadon= self.fig.add_subplot(222)
        self.AxesReconstruction= self.fig.add_subplot(223)
        
        self.AxesInput.set_xscale("linear")
        self.AxesInput.set_xlabel("X")
        self.AxesInput.set_ylabel("Y")
        self.AxesInput.set_title("Phantom")
        
        self.AxesRadon.set_xscale("linear")
        self.AxesRadon.set_xlabel("Projection axis")
        self.AxesRadon.set_ylabel("Intensity")
        self.AxesRadon.set_title("Projection")
        
        self.AxesReconstruction.set_xscale("linear")
        self.AxesReconstruction.set_xlabel("X")
        self.AxesReconstruction.set_ylabel("Y")
        self.AxesReconstruction.set_title("Reconstruction")

    def PlotInput(self,img):
        self.AxesInput.imshow(img,cmap='gray')
        self.AxesInput.set_xlim(-1,1)
        self.AxesInput.set_ylim(-1,1)
    def PlotA(self,img):
        self.AxesRadon.plot(img)

    def PlotB(self,img):
        self.AxesReconstruction.imshow(img,cmap='gray')
        self.AxesReconstruction.set_xlim(-1,1)
        self.AxesReconstruction.set_ylim(-1,1)
            

    #Callbacks

    def change_graph_button_call(self):
        self.Ev= PLOT_BUTTON_EV

    def change_domain_of_graph_call(self):
        self.Ev= TIME_FREQ_BUTTON_EV

    def graph_button_call(self):
        self.Ev = GRAPH_BUTTON_EV

    def input_change_callback(self,*args):
        self.input_changed = True
        self.Ev = INPUT_CHANGED_EV

    def add_ellipse_button_call(self):
        self.add_ellipse = True
        self.remove_ellipse = False
        self.Ev = ELLIPSE_EV
        
    def remove_ellipse_button_call(self):
        self.remove_ellipse = True
        self.add_ellipse = False
        self.Ev = ELLIPSE_EV
        
    def on_closing(self):
        if messagebox.askokcancel("Cerrar", "Desea cerrar el programa?"):
            self.Ev=QUIT_EV
    #Extras

    def CloseGUI(self):
        self.root.destroy()
    def Update(self):
        self.Graph.draw()
        self.root.update()
    def ShowMessage(self,string):
         messagebox.showinfo("",string)

    def EventSolved(self):
        self.Ev = NO_EV



In [2]:

#Estados
TIEMPO=0
FRECUENCIA=1
EXIT=2
#Constantes
FC_MAX = 6e6
class Manager(object):
    """Clase que se encarga de manejar los eventos generados"""
    def __init__(self,GUI,data,calculator):
        self.GUI=GUI
        self.Data= data
        self.calc= calculator
        self.estado= TIEMPO
    def Dispatch(self, ev):
            if(ev == NO_EV):
                self.OnNoEv()
            elif(ev == QUIT_EV):
                self.OnQuitEv()
            elif(ev == GRAPH_BUTTON_EV):
                self.OnPlotEv()
            elif(ev == PLOT_BUTTON_EV):
                self.OnPlotEv()
            elif(ev == ELLIPSE_EV):
                self.OnEllipseEv()
            else:
                self.Error()

            self.GUI.EventSolved()    #Settea que ya no hay evento a resolver


    #Getters
    def getState(self):
        return self.estado


    #Funciones que manejan eventos

    def OnNoEv(self):
        return

    def OnQuitEv(self):
        self.GUI.CloseGUI()
        self.estado=EXIT
        return
    def OnEllipseEv(self):
        if (self.GUI.remove_ellipse):
            self.Data.ellipse_list.pop(-1)
        elif (self.GUI.add_ellipse):
            Intensity= self.GUI.SlideIntensity.get()
            tilt = self.GUI.SlideTilt.get()
            x_axis = self.GUI.SlideX_Axis.get()
            y_axis = self.GUI.SlideY_Axis.get()
            x_c = self.GUI.SlideX_Center.get()
            y_c = self.GUI.SlideY_Center.get()
            elipse = [Intensity, tilt, x_axis, y_axis, x_c, y_c]
            self.Data.ellipse_list.append( elipse )
        self.GUI.remove_ellipse = False
        self.GUI.add_ellipse = False
        
    def OnPlotEv(self):
        self.GUI.graphed_once = True
        if(self.GUI.input_changed):
            self.UpdateUserData()
            self.CalculateAllNodes()
        self.ShowGraph()
    
                     
    def Error(self):
        return

    def UpdateUserData(self):
        self.phi_start = self.GUI.SlideStart.get()
        self.phi_end = self.GUI.SlideEnd.get()
        self.phi_step = self.GUI.SlideStep.get()
        self.selected_phi = self.GUI.SlidePlotAngle.get()
        self.filter = self.GUI.selected_filter.get()
        self.interp = self.GUI.selected_interp.get()


    def CalculateAllNodes(self):
        el_img= self.calc.Phantom(self.Data)
        self.Data.input = np.resize(self.Data.input,new_shape=el_img.shape)
        self.Data.input = el_img
        projection,sel = self.calc.CalculateProjections(self.Data)
        #Asigno valores
        self.Data.projections = np.resize(self.Data.projections,new_shape=projection.shape)
        self.Data.sel_projection = np.resize(self.Data.sel_projection,new_shape=sel.shape)
        self.Data.projections = projection
        self.Data.sel_projection = sel
        #Reconstruccion
        reconstruction = self.calc.CalculateReconstruction(self.Data)
        self.Data.reconstruction = np.resize(self.Data.reconstruction,new_shape=reconstruction.shape)
        self.Data.reconstruction = reconstruction

    def ShowGraph(self):
        self.GUI.AxesInput.cla()
        self.GUI.AxesRadon.cla()
        self.GUI.AxesReconstruction.cla()
        
        self.GUI.PlotInput(self.Data.input)
        self.GUI.PlotA(self.Data.sel_projection)
        self.GUI.PlotB(self.Data.reconstruction)
   

In [3]:
import numpy as np
#Constantes de la plantilla del filtro
IMG_WIDTH = 250
IMG_HEIGHT = 250

class UserData(object):
    """Clase en la que se almacenan todos los parametros elegido por el usuario"""
    def __init__(self,phi_start=0,phi_end=1,phi_step=0.05,sel_phi=0,interp='linear',filter_='ramp'):
        #Variables por default
        self.phi_start = phi_start
        self.phi_end = phi_end
        self.phi_step = phi_step
        self.selected_phi = sel_phi
        self.filter = filter_
        self.interp = interp
        self.ellipse_list = []
        #Resultados
        self.input = np.zeros(shape=(10,10) )
        self.projections=np.zeros(shape=(10,10) )
        self.sel_projection=np.zeros(shape=(10,10) )
        self.reconstruction=np.zeros(shape= (10,10) )
    

In [4]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import radon, iradon
from matplotlib.patches import Ellipse
import numpy as np

class TransferCalculator(object):
    """Clase que se encarga de calcular la salida de un bloque cualquiera conociendo la entrada"""
    def __init__(self):
        return
    #Funciones que calculan el valor de la funcion en un nodo
    def Phantom(self,data):
        ellipses = data.ellipse_list
        fig = Figure(figsize=(15,15), dpi=200)
        ax = fig.add_subplot(111, aspect='equal')
        for e in ellipses:
            el = Ellipse(xy=(e[4], e[5]), width=e[2], height=e[3], angle=e[1])
            el.set_alpha(e[0])
            ax.add_artist(el)
            el.set_clip_box(ax.bbox)
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        fig.canvas.draw()
        plt.axis('off')
        plt.savefig("phantom.png", bbox_inches='tight', transparent=True)
        return 1-imread("phantom.png", as_gray=True)
        
    def CalculateProjections(self,data):
        start = data.phi_start
        stop = data.phi_end
        step = data.phi_step
        sel_phi = data.selected_phi
        img = data.input
        
        angle_arr = np.arange(start,stop+0.1,step=step)
        projections = radon(img, theta=angle_arr)
        sel_projection = radon(img, theta=[sel_phi])
        return projections, sel_projection
        
    def CalculateReconstruction(self,data):
        img = data.projections
        filter_ = data.filter
        interp = data.interp
        angles = np.arange(start=data.phi_start, stop=data.phi_end+0.1, step=data.phi_step)
        return iradon(img, theta=angles, filter=filter_, interpolation=interp)



        


In [5]:
interfaz = SimGUI()
data= UserData()
calc= TransferCalculator()
Controller= Manager(interfaz,data,calc)

while ( (Controller.getState()) != EXIT):
    interfaz.Update()
    Controller.Dispatch(interfaz.GetEvent())

C:\Users\user\anaconda3\envs\Imagenes\lib\site-packages\skimage\transform\radon_transform.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  coords = np.array(np.ogrid[:image.shape[0], :image.shape[1]])
<ipython-input-4-b8e8a9d14004>:45: FutureWarning: 'filter' is a deprecated argument name for `iradon`. It will be removed in version 0.19. Please use 'filter_name' instead.
  return iradon(img, theta=angles, filter=filter_, interpolation=interp)
